# 第十章 跟踪与运动     

## 跟踪基础    

理解物体的运动主要包含两个部分: 识别和建模.    

识别指的是在视频流后续的帧中找出之前某帧中的感兴趣物体.    

OpenCV 中提供了两种方法实现跟踪关键点: Lucas-Kanade 和 Horn-Schunk 方法, 这两种方法分别代表了通常提到的稀疏和稠密光流.   

在得到物体初步的实际位置后, 需要对物体运动进行建模.   

## 寻找角点    

有很多种局部特征可以用来进行跟踪. 实际上, 选择用于跟踪的特征点应该是独一无二的, 或者至少接近独一无二, 并且可以与另一张图像上的其他点进行参数化比较.    

一个导数值比较明显的点是图像像素变化明显的地方, 但是他可能是在边缘上,却和这个边缘上的其他点看起来一样.    

如果一个点在两个正交的方向上都有明显的导数, 则我们认为此点是独一无二的, 因此许多可跟踪的特征点都称为角点. 从直观上将, 角点(而非边缘)是一类含有足够信息且能从当前帧和下一帧都可提取出来的点.   

最普遍使用的角定义是由 Harris 提出的. Harris 角点是使用每点周围小窗口的二阶导数的自相关矩阵.

Harris 角点定义的二阶导数自相关矩阵有两个最大特征值的地方, 这说明以此点为中心的周围存在两个不同方向的纹理(或边缘), 正如实际的角点是由至少两个边缘相交于一点而产生. 之所以采用二阶导数是因为它对于一阶均匀梯度(一阶导数是常数)不产生响应(即常数一阶导数的二阶导数为 0).   

Harris 角点的另一个优点就是旋转不变性, 这样的话即使被跟踪的物体在运动中产生了旋转也还是可以继续跟踪的.   

Harris 角点的实现是将自相关矩阵的行列式值和矩阵的迹(矩阵带权重系数)相减.再将差值同预先给定的阈值进行比较. 后来 Shi 和 Tomasi 发现, 若两个特征值中较小的一个大于最小阈值, 则会得到强角点. Shi 和 Tomasi 的方法在很多情况下都可以得到比使用 Harris 方法更好的角点.    

**cv2.goodFeaturesToTrack() 函数**:    
- 查找图像中的强角点;    
- cv2.goodFeaturesToTrack(image, maxCorners, qualityLevel, minDistance[, corners[, mask[, blockSize[, useHarrisDetector[, k]]]]]) → corners;    
- corners: 输出检测到的角点.
- maxCorners: 返回的角点个数的最大值.如果实际查找到的角点个数比这个值大, 那么就返回特征更强健的角点.    
- qualityLevel: 表示一个点被认为是角点的可接受的最小特征值参数, 最小特征值响应等于是这个值和最大特征值的乘积. 常用的值为 0.1 或 0.01.       
- minDistance: 表示返回角点之间的欧式距离不小于这个值, 用于提出一些距离较近的角点.
- mask: 可选的参数, 用于指定参与角点计算的 ROI 区域. 默认是选择整个图像计算.       
- blockSize: 计算自相关矩阵时指定点的领域, 采用小窗口计算的结果要比单点(blockSize=1)计算的结果好.   
- useHarrisDetector: 若为 0, 则使用 Shi 和 Tomasi 的方法; 若为非 0, 则使用 Harris 角点.     
- k: 设置 Harris 检测器的自相关矩阵的权重系数.

函数内部执行了非极大值抑制. 函数通常用于初始化一个基于特征点的 tracker.     

输出的角点是按照特征值从大到小的顺序排列的.   


## 亚像素级角点    

如果我们图像处理的目的不是提取用于识别的特征点, 而是进行几何测量, 则通常需要更高的精度. 而函数 cv2.goodFeaturesToTrack() 只能提供简单的像素坐标值, 也就是说, 我们有时候需要实数坐标值而不是整数坐标值.     

亚像素检测方法就是一些有关峰值点位置的计算技巧. 图像测量最常用的领域为三维重建, 摄像机标定, 图像拼接以及在卫星图像中查找特定的信号(如一栋建筑的精确位置).     

**cv2.cornerSubPix() 函数**:    
- cv2.cornerSubPix(image, corners, winSize, zeroZone, criteria) → corners;    
- corners: 设定角点的初始位置, 整数值的像素位置.    
- winSize: 从像素点中心扩展的窗口边长. 如果 winSize=Size(5,5), 那么搜索窗口尺寸为 (5*2+1) x (5*2+1) = 11 x 11.   
- zeroZone: 定义死区(该区域不参与计算)窗口边长的一半. 为了避免自相关矩阵的奇异性. 如果不需要死区, 设置为 (-1,-1).     
- criteria: 角点迭代计算的结束准则.通常为迭代最大次数和迭代得到亚像素位置的精度(criteria.maxCount 或 criteria.epsilon).      



## 不变特征    
  
除了 Harris 角点和 Shi 和 Tomasi 角点之外, 还有很角点和相关局部特征点.   

SIFT(scale-invariant feature transform) 是缩放不变性特征. 它在一点处检测主要的梯度方向, 根据这个方向记录局部梯度直方图的结果. 因此, SIFT 也是旋转不变的. 

**SIFT 实现**:   
- cv2.SIFT.detect(image[, mask]) → keypoints;    
- cv2.SIFT.compute(image, keypoints[, descriptors]) → keypoints, descriptors;    
- cv2.SIFT.detectAndCompute(image, mask[, descriptors[, useProvidedKeypoints]]) → keypoints, descriptors;    
- keypoints – The input/output vector of keypoints    
- descriptors – The output matrix of descriptors. Pass cv::noArray() if you do not need them.   
- useProvidedKeypoints – Boolean flag. If it is true, the keypoint detector is not run. Instead, the provided vector of keypoints is used and the algorithm just computes their descriptors.    




## 光流   

可以将图像中的每个像素与速度关联, 或者将图像中的每个像素和连续两帧之间的位移关联. 这样得到的是稠密光流(dense optical flow), 即将图像中的每个像素都与速度相关联.    

Horn-Schunck 方法计算的就是稠密光流的速度场, 稠密光流的计算亮量大.    

而 OpenCV 中实现了一种比较简单直接的办法, 即对前后连续两帧的一个像素邻域进行匹配. 这种方法被称为"块匹配".   

介于稠密光流计算量大的缺点, 我们找到了另一种方法: 稀疏光流法. 稀疏光流的计算需要在被跟踪之前指定一组点, 如果这些点具有某种明显的特征(例如前面讲到的角点),那么跟踪就会相对稳定和可靠.    


## Lucas-Kanade 方法    

## 稠密跟踪方法   

### Horn-Schunck 方法    

### 块匹配方法


## mean-shift 和 camshift 跟踪     


## 运动模板    

## 预估器     

### Kalman 滤波器   
### 扩展 Kalman 滤波器     
### Condensation 算法    




In [ ]:
import cv2

In [1]:
import cv2

print("hello cv2")
cv2.resize

hello cv2


TypeError: Required argument 'src' (pos 1) not found